## Imports

In [1]:
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [2]:
SEED = 8642

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data

In [3]:
BATCH_SIZE = 64

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

train_dataset = datasets.FashionMNIST(root='./data', train=True,
                                      download=True, transform=transform)
val_dataset = datasets.FashionMNIST(root='./data', train=False,
                                    download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Model

In [4]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=28 * 28, hidden_size=16, output_size=10):
        super(SimpleFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, output_size)
        # self.fc1 = nn.Linear(hidden_size, output_size)
        # self.act = nn.ReLU()

    def forward(self, x):
        x = self.fc0(x)
        return x

In [5]:
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc0])

## Train

In [ ]:
hyperparams = {
    "num_epochs": 64,
    "metric": AbsGradientEdgeMetric(nn.CrossEntropyLoss()),
    "aggregation_mode": "mean",
    "choose_thresholds": {"fc0": 0.5},
    "replace_layers": ["fc0"],
    "threshold": 0.05,
    "min_delta_epoch_replace": 8,
    "window_size": 5,
    "lr": 1e-4,
    "delete_after": 2,    
    "task_type": "classification"
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

"num_epochs: 64, metric: AbsGradientEdgeMetric, aggregation_mode: mean, choose_thresholds: {'fc0': 0.7}, replace_layers: ['fc0'], threshold: 0.004, min_delta_epoch_replace: 5, window_size: 3, lr: 0.0001, delete_after: 2, task_type: classification"

In [13]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [14]:
run = wandb.init(
    project="self-expanding-nets",
    name=f"{name}",
)

In [ ]:
criterion = nn.CrossEntropyLoss()
train_sparse_recursive(sparse_model, train_loader, train_loader, val_loader, criterion, hyperparams)

100%|██████████| 938/938 [00:06<00:00, 145.64it/s]


Epoch 1/64, Train Loss: 0.7230, Val Loss: 0.6742, Val Accuracy: 0.7719


100%|██████████| 938/938 [00:06<00:00, 156.00it/s]


Epoch 2/64, Train Loss: 0.6243, Val Loss: 0.6156, Val Accuracy: 0.7906


100%|██████████| 938/938 [00:07<00:00, 127.58it/s]


...

Epoch 13/64, Train Loss: 0.4586, Val Loss: 0.4887, Val Accuracy: 0.8326


100%|██████████| 938/938 [00:06<00:00, 153.08it/s]


Epoch 14/64, Train Loss: 0.4546, Val Loss: 0.4849, Val Accuracy: 0.8319


100%|██████████| 938/938 [00:05<00:00, 161.45it/s]


Epoch 15/64, Train Loss: 0.4506, Val Loss: 0.4814, Val Accuracy: 0.8333
Chosen edges: tensor([[  0,   0,   0,  ...,   6,   6,   6],
        [ 71, 289, 345,  ..., 746, 747, 748]]) 1277


In [ ]:
wandb.finish()

acc amount,▇▇███▆▆▆▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
del_len_choose,▄█▅▂▁▁▁▁▁
len_choose,▄█▅▂▁▁▁▁▁▂
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
params amount,▁▁▁▁▁▂▂▂▂▅▅▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████
params ratio,▁▃▃▄▅▄▄▄▅▆▇▇███████████████████████████▇
params to replace amount,█▆▅▄▃▃▄▅▆▆▆▅▄▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂
train loss,█▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train time,▁▁▁▁▁▂▂▃▂▄▄▃▃▄▅▄▅▅▄▅▅▄▅▇▆▅▅▅▅▅▅▆▆▆▆▇██▇▇
val accuracy,▁▃▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████████
val loss,█▆▅▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
